## Case 2 Experiments: equal mass BBH

In [1]:
project_path = "../../../../../../"
utils_path = project_path * "src/utils/"

data_path = project_path * "data/input/case_2/";
output_directory = project_path * "data/output/case_2/schwarzschild/";

In [2]:

cd(@__DIR__)
using Pkg; Pkg.activate(project_path); 
Pkg.instantiate();

using OrdinaryDiffEq;
using Optim;
using DiffEqSensitivity;
using Plots;
using Flux;


import Statistics: mean, sqrt
import CSV: CSV
import DataFrames: DataFrame, select, filter
import DiffEqFlux: sciml_train
import Random: seed!
import LineSearches: BackTracking
import DelimitedFiles: readdlm
import BSON: @save, @load
# gr(); # specify backend for plotting


  Activating project at `~/Escritorio/TFM/code/01_project`


In [3]:

include(utils_path * "utils.jl")
import_project_utils(utils_path);

In [4]:
# specify random seed
seed = 1234;
seed!(seed)

# script conditions
show_plots = false
save_plots_gif = false
save_data = true

# paths
test_name = "test_1_cos/"
model_name = "test_1_cos/"

output_dir = output_directory* "models/" * test_name
solutions_dir = output_dir * "solutions/"
predictions_dir = output_dir * "predictions/"
metrics_dir = output_directory * "metrics/"
img_dir = output_dir * "train_img_for_gif/"
list_directories = (output_dir, solutions_dir, metrics_dir, img_dir, predictions_dir)
create_directories(list_directories)

## Dataset

In [85]:
# time range
datasize = 1500
dt = 10.0

# Load SXS Dataset
wave_ids = [
    "SXS:BBH:0211",
    "SXS:BBH:0217"
]

println("Load SXS Dataset")
dataset = load_sxs_data(wave_ids, data_path);

Load SXS Dataset
Importing data: SXS:BBH:0211
Importing data: SXS:BBH:0217


## NN Model

In [95]:

function nn_kans_case2()
    """
    Kolmogorov-Arnold Neural Networks -> WIP
    """

    chain_chiphi = Chain(
        # x -> convert.(Float32, x), # TODO: should be done outsice chain definition
        # x -> [cos(x[1]),1/abs(x[3]),1/sqrt(abs(x[3])),sqrt(abs(x[3])),x[3],sqrt(abs(x[3]))^3,x[3]^2,x[4],x[4]^2, sin(x[1]), sin(x[1])*cos(x[1])],
        x -> [x[1], x[2], x[3], x[4], 1/x[1], 1/x[2], 1/x[3], 1/x[4]],
        kan.KAN([8, 6, 11]; grid_size=5, spline_order=3, scale_base=1.0),
        # Dense(11, 32, tanh),
        Dense(11 => 2)
    )
    chain_pe = Chain(
        # x -> convert.(Float32, x), # TODO: should be done outsice chain definition
        # x -> [1/sqrt(abs(x[3]))^3,1/abs(x[3]),1/sqrt(abs(x[3])),sqrt(abs(x[3])),x[3],sqrt(abs(x[3]))^3,x[3]^2,x[4],x[4]^2,x[3]*x[4], sqrt(abs(x[4]))],
        x -> [x[1], x[2], x[3], x[4], 1/x[1], 1/x[2], 1/x[3], 1/x[4]],
        kan.KAN([8, 6, 11]; grid_size=5, spline_order=3, scale_base=1.0),
        # Dense(11, 32, tanh),
        Dense(11 => 2)
    )

    # process parameters
    
    NN_chiphi_params, re_chiphi = Flux.destructure(chain_chiphi)
    NN_chiphi_params = NN_chiphi_params .* 0
    NN_chiphi(u, NN_chiphi_params) = re_chiphi(NN_chiphi_params)(u)
    NN_pe_params, re_pe = Flux.destructure(chain_pe)
    NN_pe_params = NN_pe_params .* 0
    NN_pe(u, NN_pe_params) = re_pe(NN_pe_params)(u)

    NN_params = vcat(NN_chiphi_params,NN_pe_params)

    return NN_params, NN_chiphi, NN_chiphi_params, NN_pe, NN_pe_params, chain_chiphi, chain_pe, re_chiphi, re_pe
end



nn_kans_case2 (generic function with 1 method)

In [96]:
println("Defining model")
global NN_params
# n_neurons = 32
# NN_params, NN_chiphi, NN_chiphi_params, NN_pe, NN_pe_params, chain_phichi, chain_pe, re_chiphi, re_pe = nn_model_case2(model_name, n_neurons, tanh)

nn_output= nn_kans_case2()
NN_params, NN_chiphi, NN_chiphi_params, NN_pe, NN_pe_params, chain_phichi, chain_pe, re_chiphi, re_pe = nn_output
l1 = length(NN_chiphi_params);

Defining model


In [97]:
dataset = add_neural_network_problem_to_dataset(dataset, nn_output);

Pre-training plot

In [98]:
example = dataset[wave_ids[1]]

nn_solution = solve(
    remake(example["nn_problem"], u0=example["u0"], p = NN_params, tspan=example["tspan"]), 
    RK4(), 
    saveat = example["tsteps"], 
    dt = example["dt_data"]
)
pred_waveform_real_train, _ = compute_waveform(
    example["dt_data"], nn_solution, example["q"], example["M"], example["model_params"]
)
plt1 = plot(
    example["tsteps"], example["true_waveform"], 
    markershape=:none, markeralpha = 0.25, 
    linewidth = 2, alpha = 0.5, label="wform data (Re)", 
    legend_position=:topleft, title= "Train progress "*wave_ids[1], 
    titlefontsize = 8, legend_font_pointsize = 6
)
plot!(
    plt1, example["tsteps"], pred_waveform_real_train, 
    markershape=:none, markeralpha = 0.25,
    linewidth = 2, alpha = 0.5, label="Waveform NN (Re)"
)

┌ Warning: Instability detected. Aborting
└ @ SciMLBase /home/rubenbalbastre/anaconda3/envs/julia/share/julia/packages/SciMLBase/VdcHg/src/integrator_interface.jl:596


BoundsError: BoundsError: attempt to access 1-element Vector{Float64} at index [2]

## callback

In [10]:

train_losses = []
train_losses_complete = []
train_metrics = []
train_metrics_complete = []
test_losses = []
test_metrics = []

callback(θ, train_loss, metrics, train_results_i, train_results_i_complete, test_results_i; show_plots = show_plots, save_plots_gif=save_plots_gif) = begin

    # list to save plots -> make a gif to project presentation
    if length(train_losses) == 0
        global plot_list = []
    end

    # unpackage training results
    N = length(train_results_i["pred_waveform"])

    # add losses
    push!(train_losses, metrics["train_loss"])
    push!(train_losses_complete, metrics["train_loss_complete"])
    push!(test_losses, metrics["test_loss"])

    # add metrics
    push!(train_metrics, metrics["train_metric"])
    push!(train_metrics_complete, metrics["train_metric_complete"])
    push!(test_metrics, metrics["test_metric"])

    if save_data

        # train waveform
        plt1 = plot(
            train_results_i_complete["tsteps"], train_results_i_complete["true_waveform"], 
            markershape=:none, label="wform data (Re)", 
            legend_position=:topleft, title= "Train progress: " * train_results_i_complete["wave_id"]
        )
        plot!(
            plt1, train_results_i_complete["tsteps"], train_results_i_complete["pred_waveform"], 
            markershape=:none, label="wform NN (Re)", legend_position=:topleft, 
            title= "Train progress: " * train_results_i_complete["wave_id"]
        )
        plot!(
            plt1, train_results_i_complete["tsteps"][1:N], train_results_i_complete["pred_waveform"][1:N], 
            markershape=:none, label="wform NN (Re)"
        )

        # test waveform
        plt12 = plot(
            test_results_i["tsteps"], test_results_i["pred_waveform"], 
            markershape=:none, label="wform data (Re)", legend=:topleft, 
            title= "Test predictions: " * test_results_i["wave_id"]
        )
        plot!(
            plt12, test_results_i["tsteps"], test_results_i["pred_waveform"], 
            markershape=:none, label="wform NN (Re)"
        )

        # p, e train
        p = train_results_i["pred_solution"][3,:]
        e = train_results_i["pred_solution"][4,:]
        plt3 = plot(train_results_i["tsteps"], p, linewidth = 2, alpha = 0.5, label="p", legend=:best, title="Train " * train_results_i_complete["wave_id"])
        plot!(twinx(), train_results_i["tsteps"], e, linewidth = 2, color=:red, alpha = 0.5, label="e", legend=:topleft)

        # p,e test
        p = test_results_i["pred_solution"][3,:]
        e = test_results_i["pred_solution"][4,:]
        plt4 = plot(test_results_i["tsteps"], p, linewidth = 2, alpha = 0.5, label="p", legend=:best, title="Test " * test_results_i["wave_id"])
        plot!(twinx(), test_results_i["tsteps"], e, linewidth = 2, color=:red, alpha = 0.5, label="e", legend=:topleft)

        # losses plot
        plt5 = plot(train_losses_complete, label="train", title="Loss functions", xlabel="Epochs")
        plot!(plt5, train_losses, label="train cost function")
        plot!(plt5, test_losses, label="test")

        # save plots
        l = @layout [[a; b] [a; b] a{0.3w}]
        plt = plot(plt1, plt12, plt3, plt4, plt5, layout=l, size=(2000, 900))
        if save_plots_gif
            push!(plot_list, plt)
        end
        if show_plots
            display(plot(plt))
        end
    end

    # Tell sciml_train to not halt the optimization. If return true, then optimization stops.
    return false
end;

## Training

In [11]:
dataset_train = Dict([key => value for (key, value) in dataset if key != wave_ids[end]]);
dataset_test = Dict([key => value for (key, value) in dataset if key == wave_ids[end]]);

In [12]:
println("Begin Progressive training...")

num_optimization_increments = 100
optimization_increments = [collect(40:10:num_optimization_increments-15)..., 85, 90, num_optimization_increments-5, num_optimization_increments-1,  num_optimization_increments]
n = length(optimization_increments)
epochs_increments = [100,100,100,100,100,100,100,100,100,150] / 10

@assert length(epochs_increments) == length(optimization_increments)

for (index, i) in enumerate(optimization_increments)

    println("optimization increment :: ", i, " of ", num_optimization_increments)
    dataset[wave_ids[1]]["tsteps"]
    tsteps_increment_bool = dataset[wave_ids[1]]["tsteps"] .<= dataset[wave_ids[1]]["tspan"][1] + i*(dataset[wave_ids[1]]["tspan"][2]-dataset[wave_ids[1]]["tspan"][1]) / num_optimization_increments
    max_epochs = round(epochs_increments[index])
    
    println("Training ", max_epochs, " epochs")

    tmp_loss(p) = loss_function_case2(
        p, 
        tsteps_increment_bool=tsteps_increment_bool, 
        dataset_train=dataset_train, 
        dataset_test=dataset_test
    )

    # to learning rates / optimization algorithm to learn at different 'stages'
    if index < n-3
        # introduce randomness to avoid local minimum (not critical)
        global NN_params = NN_params + Float64(1e-6)*randn(eltype(NN_params), size(NN_params)) #1e-7
        local res = sciml_train(
            tmp_loss, 
            NN_params,  
            BFGS(initial_stepnorm=1e-1, linesearch = BackTracking()), 
            cb=callback, 
            maxiters = max_epochs, 
            allow_f_increases=true
        )
    else
        # introduce randomness to avoid local minimum (not critical)
        global NN_params = NN_params + Float64(1e-6)*randn(eltype(NN_params), size(NN_params))
        local res = sciml_train(
            tmp_loss,
            NN_params, 
            BFGS(initial_stepnorm=1e-1, linesearch = BackTracking()), 
            cb=callback, 
            maxiters = max_epochs, 
            allow_f_increases=true
        )
    end
    
    global NN_params = res.minimizer
end
println("Finished training")


Begin Progressive training...
optimization increment :: 40 of 100
Training 10.0 epochs


┌ Warning: sciml_train is being deprecated in favor of direct usage of Optimization.jl. Please consult the Optimization.jl documentation for more details. Optimization.jl's PolyOpt solver is the polyalgorithm of sciml_train
└ @ DiffEqFlux /home/rubenbalbastre/anaconda3/envs/julia/share/julia/packages/DiffEqFlux/Em1Aj/src/train.jl:6
┌ Warning: Instability detected. Aborting
└ @ SciMLBase /home/rubenbalbastre/anaconda3/envs/julia/share/julia/packages/SciMLBase/VdcHg/src/integrator_interface.jl:596
┌ Warning: Instability detected. Aborting
└ @ SciMLBase /home/rubenbalbastre/anaconda3/envs/julia/share/julia/packages/SciMLBase/VdcHg/src/integrator_interface.jl:596
┌ Warning: Instability detected. Aborting
└ @ SciMLBase /home/rubenbalbastre/anaconda3/envs/julia/share/julia/packages/SciMLBase/VdcHg/src/integrator_interface.jl:596


BoundsError: BoundsError: attempt to access 4×2 RecursiveArrayTools.VectorOfArray{Float32, 2, Vector{Vector{Float32}}} at index [1:4, 1000-element BitVector]

## Metrics

In [12]:
niter = range(1, length(train_losses))
df_losses = DataFrame(
    epochs=niter, 
    test_name=test_name, 
    train_losses = Float64.(train_losses_complete),
    test_losses = Float64.(test_losses), 
    train_metric = Float64.(train_metrics_complete), 
    test_metric = Float64.(test_metrics)
)

Row,epochs,test_name,train_losses,test_losses,train_metric,test_metric
,Int64,String,Float64,Float64,Float64,Float64
1,1,test_1_cos/,0.000113628,0.000133126,0.132992,0.143626
2,2,test_1_cos/,0.000105132,0.000123644,0.133046,0.14365
3,3,test_1_cos/,9.28894e-5,0.000119383,0.126212,0.143093
4,4,test_1_cos/,9.76009e-5,0.000118613,0.132673,0.145484
5,5,test_1_cos/,9.52285e-5,0.000119065,0.131691,0.146452
6,6,test_1_cos/,8.97089e-5,0.00011752,0.128105,0.145575
7,7,test_1_cos/,9.17548e-5,0.000119156,0.129884,0.146653
8,8,test_1_cos/,9.29975e-5,0.000123652,0.129207,0.147494
9,9,test_1_cos/,5.51312e-5,0.000108383,0.0937773,0.120647


In [13]:
losses_plot = plot(df_losses[!, "epochs"], df_losses[!, "train_losses"], label="train")
plot!(losses_plot, df_losses[!, "epochs"], df_losses[!, "test_losses"], label="test")

ErrorException: invalid redefinition of constant losses_plot

In [14]:
plot(plot_list[end])

BoundsError: BoundsError: attempt to access 0-element Vector{Any} at index [0]

In [15]:
if save_data

    println("Saving data")

    # naming dirs
    solutions_dir = output_dir*"solutions/"
    img_dir = output_dir*"train_img_for_gif/"
    metrics_dir = output_directory*"metrics/"

    # checking if directories exist
    create_directory_if_does_not_exist(output_dir)
    create_directory_if_does_not_exist(solutions_dir)
    create_directory_if_does_not_exist(img_dir)
    create_directory_if_does_not_exist(metrics_dir)

    # save plots
    if save_plots_gif
        for (ind, img) in enumerate(plot_list)
            savefig(img, img_dir*string(ind)*"_train_img.png")
        end
    end

    # save final plot
    if show_plots
        savefig(plot_list[end], output_dir*"prediction_plot.png")
    end

    if ! isfile(metrics_dir*"losses.csv")
        CSV.write(metrics_dir*"losses.csv", df_losses)
    else
        x = DataFrame(CSV.File(metrics_dir*"losses.csv", types=Dict("test_name" => String)))
        append!(x, df_losses)
        CSV.write(metrics_dir*"losses.csv", x)
    end
end;


Saving data


## Save model

In [16]:
# https://github.com/JuliaIO/BSON.jl 
# https://stackoverflow.com/questions/66395998/saving-and-loading-model-and-the-best-weight-after-training-in-sciml-julia
# save flux chain models as bson files. To do so, we must save chain model with its parameters
NN_chiphi_params = NN_params[1:l1]
NN_pe_params = NN_params[l1+1:end]
Flux.loadparams!(chain_pe, Flux.params(re_pe(NN_pe_params)))
Flux.loadparams!(chain_phichi, Flux.params(re_chiphi(NN_chiphi_params)))
@save solutions_dir*"model_chiphi.bson" chain_phichi
@save solutions_dir*"model_pe.bson" chain_pe